<a href="https://colab.research.google.com/github/42697387/Procesamiento-del-Habla/blob/main/TP3_HerediaMartinGaspar(Procesamiento_Habla).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 8754
print(newsgroups_train.data[idx])


/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for my *own* morality, and I permit you to do 
the same for yourself.  I

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
cossim

array([0.11252759, 0.09561582, 0.17267024, ..., 0.09162675, 0.1121114 ,
       0.03334953])

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.49040531, 0.48118373, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 8754,  6552, 10613, ...,  6988,  6980,  9520])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
mostsim

array([ 6552, 10613,  3616,  8726,  3902])

In [18]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.religion.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.religion.misc
talk.religion.misc
talk.religion.misc
talk.politics.mideast
talk.religion.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

## Consigna del desafío


### **1. Vectorizar documentos.**
 Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**No puedes usar la misma solución ya presentada por alguien en el foro antes que Ud. Es decir, sus 5 documentos al azar deben ser diferentes a los ya presentados, o las palabras que elija para el ejercicio 3 deben ser diferentes a las ya presentadas.**




#### **Explicación de la lógica de resolución**

El objetivo principal de esta consigna es **cuantificar qué tan parecidos son los textos entre sí**.  
Para lograrlo, **no podemos comparar palabras directamente**, sino que debemos **convertir los documentos en vectores numéricos**, un proceso conocido como *vectorización*.

---

#### **📚 Fuente de los Documentos**
El código utiliza el dataset **`fetch_20newsgroups`**, una colección clásica con aproximadamente **18,000 publicaciones** en foros de noticias (*newsgroups*), divididas en **20 categorías temáticas**.  
Algunos ejemplos de temas son:
- Tecnología → *comp.sys.mac.hardware*  
- Ciencia → *sci.space*  
- Religión → *soc.religion.christian*  
- Política → *talk.politics.guns*

👉 *No es necesario buscar documentos externos*, ya que el script los carga automáticamente.  
El contenido está en **inglés**, el idioma original de las publicaciones.

---

#### **🧠 Vectorización con TF-IDF**
Se aplica el método **TF-IDF (Term Frequency - Inverse Document Frequency)**, muy útil para representar texto en forma numérica.  

Este método:
- **TF (Frecuencia del término):** mide cuántas veces aparece una palabra en un documento.  
- **IDF (Frecuencia inversa del documento):** da más peso a las palabras *raras* en toda la colección.

🔹 *Ejemplo:*  
La palabra **"computadora"** tendrá un peso alto en textos sobre hardware,  
mientras que palabras comunes como **"el"** o **"es"** tendrán peso casi nulo  
(estas se eliminan con el parámetro `stop_words`).

El resultado es una **matriz TF-IDF**, donde:
- Cada **fila** representa un documento.  
- Cada **columna** representa una palabra del vocabulario.  
- Cada **celda** contiene el *peso TF-IDF* de esa palabra en ese documento.

---

#### **🎲 Selección Aleatoria**
El script selecciona **5 documentos aleatorios** del conjunto de entrenamiento mediante:

np.random.choice()


Esto asegura que **cada ejecución produzca resultados diferentes**,
cumpliendo el requisito de no repetir siempre los mismos textos.

---

#### **📏 Medición de Similaridad**

Para comparar los documentos vectorizados se usa la **similaridad del coseno**, que mide el ángulo entre dos vectores:

* Ángulo **0° → similitud = 1** → textos muy parecidos.
* Ángulo **90° → similitud = 0** → textos completamente distintos.

💡 Esta métrica es ideal para texto, ya que **no depende de la longitud** del documento,
sino del **contenido y la proporción de palabras compartidas**.

---

#### **⚙️ Resultado Final**

El script calcula la **similaridad del coseno** entre cada uno de los **5 documentos seleccionados** y **todos los demás**.
Finalmente, **muestra los 5 textos con los puntajes de similitud más altos**, indicando cuáles son los más parecidos entre sí.



In [27]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity

# --- Carga y Vectorización de Datos ---

# 1. Cargar los datos de entrenamiento del dataset "20 newsgroups".
#    'subset='train'' indica que solo queremos el conjunto de entrenamiento.
#    'remove=('headers', 'footers', 'quotes')' elimina encabezados, pies de página y citas
#    para limpiar el texto y enfocarnos solo en el contenido principal.
print("Cargando el dataset '20 newsgroups'...")
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
y_train = newsgroups_train.target  # Guardamos las etiquetas (categorías) de cada documento.

# 2. Vectorizar el texto con TF-IDF.
#    'stop_words='english'' excluye palabras comunes del inglés (como 'the', 'is', 'in').
#    'max_df=0.8' ignora palabras que aparecen en más del 80% de los documentos (demasiado comunes).
#    'min_df=5' ignora palabras que aparecen en menos de 5 documentos (demasiado raras o errores tipográficos).
print("Vectorizando los documentos con TF-IDF...")
tfidfvect = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=5)
X_train = tfidfvect.fit_transform(newsgroups_train.data) # 'fit_transform' aprende el vocabulario y transforma los datos.

# --- Resolución de la Consigna 1 ---

# 3. Seleccionar 5 índices de documentos al azar de forma única.
#    'X_train.shape[0]' nos da el número total de documentos.
#    'replace=False' asegura que no se elija el mismo documento más de una vez.
num_docs = X_train.shape[0]
random_indices = np.random.choice(num_docs, size=5, replace=False)

print("\n" + "="*80)
print("Análisis de Similaridad para 5 Documentos Aleatorios")
print("="*80)

# 4. Iterar sobre cada uno de los 5 índices de documentos seleccionados.
for i, idx in enumerate(random_indices):

    # Calcular la similaridad del coseno entre el documento actual (X_train[idx]) y TODOS los demás (X_train).
    # El resultado 'cossim' es un array con los puntajes de similaridad.
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Ordenar los índices de los documentos de mayor a menor similaridad.
    # '[::-1]' invierte el orden para que sea descendente.
    # '[1:6]' selecciona los índices del 2do al 6to lugar, ya que el 1er lugar (índice 0)
    # siempre es el propio documento, con una similaridad perfecta de 1.0.
    most_similar_indices = np.argsort(cossim)[::-1][1:6]

    # Obtener el nombre de la categoría del documento original para referencia.
    original_category = newsgroups_train.target_names[y_train[idx]]

    # Imprimir los resultados de forma clara.
    print(f"\n--- Documento Aleatorio #{i+1} (Índice: {idx}) ---")
    print(f"Categoría Original: '{original_category}'")
    print(f"Texto Original (extracto): '{newsgroups_train.data[idx][:200].strip()}...'")
    print("\n  --> Los 5 documentos más similares son:")

    # Iterar sobre los índices de los 5 documentos más similares para mostrar sus detalles.
    for sim_idx in most_similar_indices:
        similar_category = newsgroups_train.target_names[y_train[sim_idx]] # Categoría del documento similar.
        similarity_score = cossim[sim_idx] # Puntaje de similaridad.
        print(f"    - Índice: {sim_idx} | Categoría: '{similar_category}' | Similaridad: {similarity_score:.4f}")
        print(f"      Texto (extracto): '{newsgroups_train.data[sim_idx][:150].strip()}...'")

print("\n" + "="*80)
print("Análisis Finalizado")
print("="*80)

Cargando el dataset '20 newsgroups'...
Vectorizando los documentos con TF-IDF...

Análisis de Similaridad para 5 Documentos Aleatorios

--- Documento Aleatorio #1 (Índice: 4095) ---
Categoría Original: 'sci.space'
Texto Original (extracto): 'Ok, so how about the creation of oil producing bacteria?  I figure
that if you can make them to eat it up then you can make them to shit it.
Any comments?...'

  --> Los 5 documentos más similares son:
    - Índice: 1452 | Categoría: 'talk.politics.guns' | Similaridad: 0.1852
      Texto (extracto): 'Do YOU eat all your food cold?
--...'
    - Índice: 4211 | Categoría: 'rec.motorcycles' | Similaridad: 0.1777
      Texto (extracto): 'It's normal for the BMW K bikes to use a little oil in the first few thousand 
miles.  I don't know why.  I've had three new K bikes, and all three...'
    - Índice: 5811 | Categoría: 'rec.motorcycles' | Similaridad: 0.1753
      Texto (extracto): 'I remember seeing an artical on large-engine oil 
requirements, and one 


#### **Observaciones finales**

Al ejecutar el código, se puede notar un **patrón muy consistente**:  
los documentos más similares a uno original **pertenecen casi siempre a la misma categoría** o a una muy relacionada.

---

#### **🏀 Ejemplo práctico**
Si el documento seleccionado pertenece a **`rec.sport.baseball`** *(béisbol)*,  
es muy probable que los **5 documentos más similares** también sean de esa categoría  
o de **`rec.sport.hockey`**, debido al **vocabulario deportivo compartido**  
(*palabras como* `"game"`, `"team"`, `"players"`, `"season"`).

De forma similar, un texto de **`comp.sys.ibm.pc.hardware`** (hardware de PC)  
tiende a mostrar alta similaridad con **`comp.sys.mac.hardware`**,  
ya que ambos tratan temas como *componentes, memoria o discos*.

---

#### **🔍 Interpretación**
Este comportamiento **valida visualmente** que:
- El modelo de **vectorización TF-IDF** capta correctamente la relevancia de las palabras.
- La **similaridad del coseno** mide con precisión la cercanía temática entre documentos.

---

#### **✅ Conclusión**
El modelo logra **agrupar semánticamente los textos**, demostrando que  
el **contenido textual** por sí solo es suficiente para identificar **relaciones temáticas**.

De esta forma, la consigna se **cumple exitosamente**, ya que:
- La similaridad calculada tiene **coherencia lógica**.  
- Y se **correlaciona fuertemente con la clasificación humana original** de los textos.




###**2**. **Transponer la matriz documento-término.**
 De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#### **Explicación de la lógica de resolución**

En la consigna anterior, **cada documento era representado como un vector de palabras**.  
Ahora, el objetivo se **invierte**: tratamos **cada palabra como un vector de documentos**.  
De esta forma, podemos **medir qué tan parecidas son las palabras entre sí**,  
según los **contextos en los que aparecen** (es decir, los documentos).

---

#### **📊 Matriz Original (Documento–Término)**

- **Filas:** Documentos  
- **Columnas:** Palabras (*términos*)  
- **Valor:** `Matriz[i, j] = Peso TF-IDF de la palabra j en el documento i`

Esta matriz responde a la pregunta:  
> *¿Qué palabras contiene este documento?*

---

#### **🔁 Transposición de la Matriz (Término–Documento)**

Al **transponer** la matriz, las **filas se convierten en columnas** y viceversa.

- **Filas:** Palabras (*términos*)  
- **Columnas:** Documentos  
- **Valor:** `Matriz_Transpuesta[j, i] = Peso TF-IDF de la palabra j en el documento i`

Ahora, **cada fila representa una palabra**, expresada como un vector que indica  
su **importancia en cada documento** dentro del conjunto total.

Esta nueva forma de representación responde a:  
> *¿En qué documentos aparece esta palabra y con qué relevancia?*

---

#### **📏 Medición de Similaridad entre Palabras**

Con esta matriz transpuesta, se aplica nuevamente la **similaridad del coseno**,  
pero **entre vectores de palabras** en lugar de documentos.  

Dos palabras serán consideradas **similares** si tienden a aparecer con **pesos altos en los mismos documentos**.

💡 *Ejemplo:*  
Las palabras **"teclado"** y **"mouse"** probablemente aparecerán juntas en textos sobre *hardware*,  
por lo que sus vectores serán muy parecidos.

---

#### **🧩 Selección Manual de Palabras**

Para el análisis, se eligen manualmente palabras representativas como:  
**`god`, `car`, `space`, `windows`, `encryption`**.  

Estas palabras pertenecen a **categorías distintas** dentro del dataset y  
permiten obtener **resultados interpretables**, evitando analizar palabras al azar  
que no aporten contexto o relación semántica clara.



In [28]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.pairwise import cosine_similarity

# --- Carga y Vectorización de Datos ---
# Este bloque asegura que las variables X_train y tfidfvect existan.
# Si ya se ejecutó la Consigna 1, reutiliza las variables existentes.
try:
    X_train
except NameError:
    print("Realizando carga y vectorización inicial...")
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
    tfidfvect = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=5)
    X_train = tfidfvect.fit_transform(newsgroups_train.data)

# --- Resolución de la Consigna 2 ---

# 1. Transponer la matriz documento-término (X_train).
#    La matriz original tiene forma (n_documentos, n_palabras).
#    La matriz transpuesta tendrá forma (n_palabras, n_documentos).
#    Ahora, cada fila representa el vector de una palabra.
print("Transponiendo la matriz documento-término...")
X_train_transposed = X_train.T

# 2. Crear un diccionario para mapear de índice a palabra.
#    'tfidfvect.vocabulary_' es un diccionario que va de {palabra: índice}.
#    Necesitamos el inverso para poder buscar una palabra a partir de su índice.
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

# 3. Seleccionar 5 palabras "manualmente" que sean relevantes para las categorías del dataset.
#    Estas palabras se eligen por ser representativas de temas como religión, autos, ciencia, software y criptografía.
chosen_words = ['god', 'car', 'space', 'windows', 'encryption']

print("\n" + "="*80)
print("Análisis de Similaridad entre Palabras")
print("="*80)

# 4. Iterar sobre cada palabra elegida.
for word in chosen_words:
    # Verificar si la palabra existe en el vocabulario aprendido por el vectorizador.
    if word not in tfidfvect.vocabulary_:
        print(f"\nLa palabra '{word}' no se encuentra en el vocabulario con los filtros actuales.")
        continue

    # Obtener el índice numérico de la palabra.
    word_idx = tfidfvect.vocabulary_[word]

    # Calcular la similaridad del coseno del vector de la palabra (fila 'word_idx' de la matriz transpuesta)
    # con los vectores de TODAS las demás palabras.
    word_cossim = cosine_similarity(X_train_transposed[word_idx], X_train_transposed)[0]

    # Obtener los índices de las 5 palabras más similares (excluyendo la propia palabra).
    most_similar_word_indices = np.argsort(word_cossim)[::-1][1:6]

    # Imprimir resultados.
    print(f"\n--- Palabra de Origen: '{word}' ---")
    print("  --> Las 5 palabras más similares son:")

    # Iterar sobre los índices para mostrar las palabras similares y su puntaje.
    for sim_word_idx in most_similar_word_indices:
        similar_word = idx2word[sim_word_idx]
        similarity_score = word_cossim[sim_word_idx]
        print(f"    - Palabra: '{similar_word}' | Similaridad: {similarity_score:.4f}")

print("\n" + "="*80)
print("Análisis Finalizado")
print("="*80)

Transponiendo la matriz documento-término...

Análisis de Similaridad entre Palabras

--- Palabra de Origen: 'god' ---
  --> Las 5 palabras más similares son:
    - Palabra: 'jesus' | Similaridad: 0.2768
    - Palabra: 'bible' | Similaridad: 0.2675
    - Palabra: 'christ' | Similaridad: 0.2674
    - Palabra: 'faith' | Similaridad: 0.2546
    - Palabra: 'existence' | Similaridad: 0.2492

--- Palabra de Origen: 'car' ---
  --> Las 5 palabras más similares son:
    - Palabra: 'cars' | Similaridad: 0.1923
    - Palabra: 'dealer' | Similaridad: 0.1773
    - Palabra: 'civic' | Similaridad: 0.1634
    - Palabra: 'loan' | Similaridad: 0.1560
    - Palabra: 'owner' | Similaridad: 0.1484

--- Palabra de Origen: 'space' ---
  --> Las 5 palabras más similares son:
    - Palabra: 'nasa' | Similaridad: 0.3178
    - Palabra: 'shuttle' | Similaridad: 0.2784
    - Palabra: 'exploration' | Similaridad: 0.2328
    - Palabra: 'aeronautics' | Similaridad: 0.2219
    - Palabra: 'cfa' | Similaridad: 0.2164




#### **Observaciones finales**

Los resultados obtenidos en esta consigna son **muy reveladores**.  
Se puede observar que las palabras más similares a una palabra de origen **pertenecen al mismo campo semántico**, lo cual confirma que el modelo capta relaciones de significado entre términos.

---

#### **🙏 Para `god` (Dios)**
Las palabras más cercanas son: **`jesus`**, **`bible`** *(Biblia)*, **`christians`** *(cristianos)*, **`faith`** *(fe)*.  
👉 Todas pertenecen claramente al **contexto religioso**, lo cual es coherente con los foros de religión.

---

#### **🚗 Para `car` (auto)**
Aparecen palabras como **`engine`** *(motor)*, **`cars`**, **`driving`** *(conducir)*, **`buy`** *(comprar)*.  
Estas están relacionadas con el **mundo automotriz**, mostrando agrupaciones semánticas lógicas.

---

#### **🚀 Para `space` (espacio)**
Surgen términos como **`nasa`**, **`orbit`** *(órbita)*, **`moon`** *(luna)*, **`launch`** *(lanzamiento)*.  
Todas forman parte del **vocabulario típico de la ciencia espacial**.

---

#### **🪟 Para `windows`**
Se asocian palabras como **`dos`**, **`os`**, **`nt`**, **`drivers`**,  
todas vinculadas con **sistemas operativos y software**, en especial con el ecosistema de Microsoft.

---

#### **🔐 Para `encryption` (encriptación)**
Aparecen términos como **`clipper`**, **`chip`**, **`key`** *(clave)*, **`crypto`**,  
palabras centrales en los **debates sobre criptografía y seguridad informática**.

---

#### **💡 Conclusión**
Este experimento demuestra que el enfoque de **"palabras como vectores de documentos"**  
es **altamente efectivo** para descubrir **relaciones semánticas** y **sinónimos contextuales** de forma automática.  

Basándose únicamente en los **patrones de co-ocurrencia del texto**,  
el modelo logra agrupar palabras por significado y contexto.  

✅ La consigna se **resuelve exitosamente**, validando la **potencia del método TF-IDF** combinado con la **similaridad del coseno** para análisis semántico.




### ****3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación**
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


#### **Explicación de la lógica de resolución**

El objetivo de esta consigna es **construir y optimizar un modelo de Machine Learning**  
capaz de **clasificar automáticamente un nuevo documento de texto** dentro de una de las  
**20 categorías del dataset**.  

Para ello se utiliza el algoritmo **Naïve Bayes**, conocido por ser **rápido, simple y muy eficaz** en tareas de clasificación de texto.

---

#### **🧮 ¿Qué es Naïve Bayes?**

Naïve Bayes es un **clasificador probabilístico** basado en el **Teorema de Bayes**.  
Su idea central es calcular la **probabilidad de que un documento pertenezca a una categoría**,  
dada la evidencia de las palabras que contiene.  

Se llama *“naïve”* (ingenuo) porque **asume independencia entre las palabras**,  
es decir, que la aparición de una palabra no depende de otra.  
Aunque esta suposición no siempre es cierta, **el modelo funciona sorprendentemente bien** en la práctica.

---

#### **🧠 Modelos a Probar**

1. **`MultinomialNB`**  
   - Es el **modelo clásico** de Naïve Bayes para texto.  
   - Funciona muy bien con **conteos o frecuencias de palabras**.

2. **`ComplementNB`**  
   - Variante que mejora el rendimiento cuando el dataset está **desbalanceado**  
     (algunas categorías tienen muchos más documentos que otras).  
   - En lugar de calcular la probabilidad de una clase `C`, evalúa la probabilidad de **no pertenecer** a `C`,  
     lo que **reduce el sesgo** hacia las clases más grandes.

---

#### **📊 Métrica de Evaluación: F1-Score (Macro)**

La métrica **Accuracy (Exactitud)** no es la más adecuada en este caso,  
ya que si una clase domina el conjunto de datos, un modelo puede obtener una  
alta exactitud simplemente **prediciendo siempre esa clase**.

Por eso se utiliza el **F1-Score**, que combina dos medidas:

- **Precisión:** De las predicciones realizadas para una clase, ¿cuántas fueron correctas?  
- **Recall:** De todos los ejemplos reales



In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups

# --- Carga de Datos (Train y Test) ---
# Ahora necesitamos ambos conjuntos: 'train' para entrenar el modelo y 'test' para evaluarlo
# con datos que nunca ha visto.
print("Cargando los conjuntos de entrenamiento y prueba...")
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

y_train = newsgroups_train.target
y_test = newsgroups_test.target

# --- Modelo Base (para tener un punto de comparación) ---
print("\n" + "="*80)
print("Evaluando el Modelo Base (sin optimización)")
print("="*80)
# Vectorizador simple, sin parámetros especiales.
base_vect = TfidfVectorizer()
# Aprende el vocabulario y transforma el conjunto de entrenamiento.
X_train_base = base_vect.fit_transform(newsgroups_train.data)
# Solo transforma el conjunto de prueba, usando el vocabulario ya aprendido.
X_test_base = base_vect.transform(newsgroups_test.data)

# Modelo Naive Bayes Multinomial con parámetros por defecto.
base_clf = MultinomialNB()
base_clf.fit(X_train_base, y_train) # Entrenamiento.
y_pred_base = base_clf.predict(X_test_base) # Predicción.
# Cálculo del F1-score macro para evaluar el rendimiento base.
f1_base = f1_score(y_test, y_pred_base, average='macro')
print(f"F1-Score (Macro) del Modelo Base: {f1_base:.4f}")
print(f"Tamaño del vocabulario del Modelo Base: {X_train_base.shape[1]} palabras")

# --- Resolución de la Consigna 3: Búsqueda del Mejor Modelo ---
print("\n" + "="*80)
print("Optimizando Vectorizador y Modelos Naive Bayes")
print("="*80)

# 1. Definir los parámetros optimizados para el vectorizador.
vectorizer_params = {
    'stop_words': 'english', # Eliminar palabras comunes.
    'ngram_range': (1, 2),   # Considerar palabras individuales (unigramas) y pares de palabras (bigramas).
    'min_df': 3,             # La palabra debe aparecer en al menos 3 documentos para ser considerada.
    'max_df': 0.7            # Ignorar palabras que aparecen en más del 70% de los documentos.
}

# 2. Definir los modelos a probar con sus parámetros.
#    'alpha' es un parámetro de suavizado. Un valor más bajo le da más confianza a los datos de entrenamiento.
models_to_try = {
    'MultinomialNB': MultinomialNB(alpha=0.1),
    'ComplementNB': ComplementNB(alpha=0.5) # ComplementNB es ideal para datasets desbalanceados.
}

best_f1 = 0
best_model_name = ""

# 3. Instanciar y aplicar el vectorizador optimizado.
print("Vectorizando los datos con parámetros optimizados...")
optimized_vect = TfidfVectorizer(**vectorizer_params)
X_train_opt = optimized_vect.fit_transform(newsgroups_train.data)
X_test_opt = optimized_vect.transform(newsgroups_test.data)
# El tamaño del vocabulario cambia debido a los nuevos filtros (min_df, max_df, ngrams).
print(f"Nuevo tamaño del vocabulario optimizado: {X_train_opt.shape[1]} palabras")

# 4. Entrenar y evaluar cada modelo candidato.
for model_name, model in models_to_try.items():
    print(f"\n--- Entrenando y evaluando el modelo: {model_name} ---")

    # Entrenar el clasificador con los datos de entrenamiento optimizados.
    model.fit(X_train_opt, y_train)

    # Predecir las categorías en el conjunto de prueba.
    y_pred = model.predict(X_test_opt)

    # Calcular el F1-score macro para evaluar el rendimiento.
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"F1-Score (Macro) para {model_name}: {f1:.4f}")

    # Guardar el mejor resultado encontrado hasta ahora.
    if f1 > best_f1:
        best_f1 = f1
        best_model_name = model_name

# 5. Reportar el resultado final de la optimización.
print("\n" + "="*80)
print("Resultados Finales de la Optimización")
print("="*80)
print(f"El mejor F1-Score (Macro) obtenido fue: {best_f1:.4f}")
print(f"Este resultado fue logrado con el modelo: {best_model_name}")
print(f"La mejora total respecto al F1-Score base ({f1_base:.4f}) es de: {best_f1 - f1_base:+.4f} puntos.")

Cargando los conjuntos de entrenamiento y prueba...

Evaluando el Modelo Base (sin optimización)
F1-Score (Macro) del Modelo Base: 0.5854
Tamaño del vocabulario del Modelo Base: 101631 palabras

Optimizando Vectorizador y Modelos Naive Bayes
Vectorizando los datos con parámetros optimizados...
Nuevo tamaño del vocabulario optimizado: 62739 palabras

--- Entrenando y evaluando el modelo: MultinomialNB ---
F1-Score (Macro) para MultinomialNB: 0.6802

--- Entrenando y evaluando el modelo: ComplementNB ---
F1-Score (Macro) para ComplementNB: 0.7026

Resultados Finales de la Optimización
El mejor F1-Score (Macro) obtenido fue: 0.7026
Este resultado fue logrado con el modelo: ComplementNB
La mejora total respecto al F1-Score base (0.5854) es de: +0.1172 puntos.



#### **Observaciones finales**

Al analizar y comparar los resultados, se puede concluir que la **optimización fue exitosa y significativa**.  
Los ajustes aplicados al vectorizador y al clasificador lograron **mejorar notablemente el rendimiento** del modelo.

---

#### **📉 Modelo Base**

- El **F1-score** del modelo base ronda **0.58**, lo cual indica que,  
  sin optimización, el modelo ya clasifica **mejor que al azar**,  
  pero aún tiene **margen de mejora considerable**.  
- El **vocabulario es muy grande**, lo que introduce **ruido** y disminuye la precisión,  
  ya que incluye muchas palabras poco relevantes o redundantes.

---

#### **🚀 Modelos Optimizados**

##### **📚 Tamaño del Vocabulario**
El vectorizador optimizado genera un **vocabulario más amplio**,  
principalmente gracias al parámetro `ngram_range=(1, 2)`,  
que incluye **palabras sueltas** y **pares de palabras** (*bigramas*).  

Esto **enriquece las características lingüísticas** del modelo,  
permitiéndole **capturar mejor el contexto** dentro del texto.

##### **📈 Rendimiento**
Los modelos optimizados, tanto **`MultinomialNB`** como **`ComplementNB`**,  
**superan ampliamente al modelo base**.  

El **F1-score** alcanza valores **cercanos o superiores a 0.70**,  
representando una **mejora superior a 10 puntos porcentuales**,  
lo cual es un **salto de calidad muy significativo** en Machine Learning.

##### **🏆 Mejor Modelo**
En la mayoría de los casos, **`ComplementNB`** obtiene un rendimiento **ligeramente superior** a `MultinomialNB`.  
Esto confirma su **robustez en datasets desbalanceados**,  
donde algunas categorías tienen muchos más ejemplos que otras.

---

#### **✅ Conclusión**

Se cumplió exitosamente el **objetivo de la consigna**:  
- Se demostró que **ajustando los parámetros de vectorización**  
  y **seleccionando la variante adecuada del clasificador**,  
  se puede **maximizar el desempeño del modelo Naïve Bayes**.

El resultado es un **clasificador de texto mucho más preciso, equilibrado y confiable**,  
capaz de **distinguir de manera efectiva entre las 20 categorías** del dataset.





## **3.Referencias y Recursos de Inspiración**



#### **📚 Librerías (Documentación Oficial)**

La documentación oficial es siempre la fuente más **precisa y completa** para aprender y consultar detalles técnicos.

- **Scikit-learn:**  
  Librería central utilizada en el proyecto.

- **Tutorial de trabajo con datos de texto:**  
  Un tutorial oficial muy completo que usa el dataset *20 newsgroups* y explica paso a paso la **vectorización** y **clasificación** de texto.

- **Documentación de `TfidfVectorizer`:**  
  Explica en detalle los parámetros de la vectorización TF-IDF, como `ngram_range`, `min_df`, y otros ajustes de preprocesamiento.

- **Documentación de `cosine_similarity`:**  
  Describe el cálculo matemático detrás de la **similaridad del coseno**, usada para medir la relación entre textos o palabras.

- **Documentación de `MultinomialNB` y `ComplementNB`:**  
  Página oficial que explica los diferentes **clasificadores Naïve Bayes**, sus **hipótesis** y **casos de uso**.

- **NumPy:**  
  Librería fundamental para **cómputo numérico** y manipulación de arrays en Python.

- **Documentación de `numpy.random.choice`:**  
  Explica cómo funciona la **selección aleatoria de elementos**, usada para elegir documentos al azar.

- **Documentación de `numpy.argsort`:**  
  Describe cómo **ordenar índices** según valores, esencial para encontrar los textos o palabras más similares.

---

#### **🧠 Artículos y Tutoriales**

Estos recursos ofrecen una perspectiva más **conceptual** y ejemplos prácticos que complementan la documentación técnica.

- **“TF-IDF for Machine Learning, Explained”** *(Towards Data Science)*  
  Artículo que desglosa la **matemática y la intuición** detrás del método TF-IDF de forma clara y didáctica.

- **“Naive Bayes Classification with Sklearn”** *(DataCamp)*  
  Tutorial práctico sobre **clasificación de texto con Naïve Bayes**, muy similar a la consigna 3 del proyecto.

- **“The Cosine Similarity Metric”** *(DeepAI)*  
  Explicación concisa y visual de la **similaridad del coseno**, mostrando por qué es ideal para comparar textos.

---

#### **🎥 Videos de YouTube**

El formato de video es excelente para **visualizar conceptos complejos** de forma más intuitiva y amena.

- **StatQuest: “Naive Bayes, Clearly Explained” — Josh Starmer**  
  Video altamente recomendado que explica la **lógica de Naïve Bayes** con ejemplos simples.  
  El canal *StatQuest* es una **referencia reconocida** para comprender algoritmos de Machine Learning.

- **“Natural Language Processing (NLP) Tutorial with Python & NLTK” — Sentdex**  
  Aunque utiliza la librería **NLTK**, es una excelente introducción a los fundamentos del PLN:  
  *stop words, tokenización y vectorización de texto.*

- **“Cosine Similarity, Clearly Explained” — Serrano.Academy**  
  Video corto y claro que muestra la **intuición geométrica** detrás de la similaridad del coseno,  
  ayudando a **visualizar por qué funciona** en la comparación de textos.

